In [1]:
%run "../scripts/outlier.py" "../scripts/paths.json"

In [ ]:
# added fraud_feature0
# NLP - 1


In [2]:
# Reading in segments 
segments_data = spark.read.csv('..data/curated/tagged_merchants.csv').na.drop(subset=["_c0"])



AnalysisException: Path does not exist: file:/C:/Users/taiya/OneDrive/Documents/GitHub/generic-buy-now-pay-later-project-group-10-bnpl/notebooks/..data/curated/tagged_merchants.csv

In [ ]:
final_dataset.createOrReplaceTempView("temp")
segments_data.createOrReplaceTempView("temp2")

final_join = spark.sql(""" 

SELECT *
FROM temp


LEFT JOIN temp2

ON temp.merchant_abn = temp2._c3
""")

In [ ]:
#Creating a table of average fraud probabilities for each category of purchase: 

aggregated_prob = final_join.groupBy("temp2._c6").agg(avg("cons_fraud_prob").alias("avg_consumer_fraud"),avg("merch_fraud_prob").alias("avg_merchant_fraud"))
aggregated_prob.limit(5)

In [ ]:
#adding a column for is_fraud for each transaction: 

is_fraud = when((final_join["_c6"] == "Electronics") & (final_join["cons_fraud_prob"] > 15.085) &(final_join["merch_fraud_prob"]>29.635),1)\
.when((final_join["_c6"] == "Toys and DIY") & (final_join["cons_fraud_prob"] > 15.843) &(final_join["merch_fraud_prob"]>32.404),1)\
.when((final_join["_c6"] == "Furniture") & (final_join["cons_fraud_prob"] > 15.077) & (final_join["merch_fraud_prob"]>30.941),1)\
.when((final_join["_c6"] == "Beauty, Health, Personal and Household") & (final_join["cons_fraud_prob"] > 15.392) & (final_join["merch_fraud_prob"]>29.76),1)\
.when((final_join["_c6"] == "Electronics") & (final_join["cons_fraud_prob"] > 15.085) &(final_join["merch_fraud_prob"]>29.635),1)\
.when((final_join["_c6"] == "Books, Stationary and Music") & (final_join["cons_fraud_prob"] > 15.140) &(final_join["merch_fraud_prob"]>29.016),1).otherwise(0)
model_with_fraud = final_join.withColumn("is_fraud",is_fraud)

In [ ]:
#Creating a table of average fraud probabilities for each category of purchase: 
final_model_with_fraud = model_with_fraud.groupBy("merchant_abn_1").agg(avg("is_fraud").alias("average fraud rate per merchant"))
final_model_with_fraud.limit(20)

In [ ]:


# avg of prob for each segments
# label(bool) each transaction based on the threshold
# avg label for each merchant